In [2]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
import string
import re
from sklearn.ensemble import RandomForestRegressor
import itertools
from sklearn.ensemble import AdaBoostRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold # import KFold



In [3]:
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_columns',1000)


In [4]:
folder = '/Users/jessyjinshu/CS506-Final-Project/'
#the position of the dataset folder

In [5]:
data = pd.read_csv(folder+'pc_spark.csv')
X_train = pd.read_csv(folder+'X_train.csv',index_col=0)
X_test = pd.read_csv(folder+'X_test.csv',index_col=0)
y_train = pd.read_csv(folder+'y_train.csv',index_col=0)
y_test = pd.read_csv(folder+'y_test.csv',index_col=0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
X = pd.merge(X_train,y_train,right_index=True, left_index=True)
y = pd.merge(X_test,y_test,right_index=True, left_index=True)
training = pd.concat([X,y])

In [8]:
# training['log_price_per_ha'] = np.log(training['ls_price_pc']/training['ha'])
training = training.merge(training.groupby('fips')['ls_price_pc'].mean().reset_index(name='fip_avg'),how='left')
training['fip_avg'] = training['fip_avg'].fillna(training['fip_avg'].mean()) #compute the average price within county
training = training.loc[training['year'].isnull()==False].copy() 
training.columns

Index(['p_water', 'river_frontage', 'f_year', 'p_prot_2000_5000',
       'p_prot_1990_5000', 'p_forest_m', 'pid', 'gisjoin', 'slope', 'p_f',
       'p_prot_2010_1000', 'p_dev_open', 'p_dev_medium', 'lid', 'p_shrub',
       'lake_importance', 'p_forest_e', 'p_prot_1990_200', 'ha', 'e_year',
       'p_prot_2010_5000', 'p_dev_high', 'csd_id', 'travel', 'p_dev_low',
       'p_wetland_w', 'p_prot_2000_1000', 'e_orgtype', 'p_grassland', 'fips',
       'p_prot_2000_200', 'p_wetland_e', 'p_prot', 'p_prot_1990_1000',
       'p_prot_2010_200', 'ls_price_pc', 'p_pasture', 'p_e', 'ls_date_pc',
       'p_forest_d', 'owner_pc', 'p_barren', 'f_orgtype', 'lake_frontage',
       'p_crops', 'p_wet', 'pop_dens_bg_1990', 'pop_dens_bg_2000',
       'pop_dens_bg_2006-2010', 'pop_dens_bg_2012-2016', 'pop_dens_tract_1990',
       'pop_dens_tract_2000', 'pop_dens_tract_2006-2010',
       'pop_dens_tract_2012-2016', 'hh_inc_med_bg_1990', 'hh_inc_med_bg_2000',
       'hh_inc_med_bg_2006-2010', 'hh_inc_med_bg_201

In [ ]:
#attributes in the model
input_list_resource = ['p_water', 'river_frontage', 'p_forest_m', 'p_f',
                       'p_dev_open', 'p_dev_medium',  'p_shrub',
                       'lake_importance', 'p_forest_e', 'p_dev_high', 'p_dev_low',
                       'p_wetland_w', 'p_grassland', 'p_wetland_e', 'p_prot','p_pasture', 
                       'p_forest_d',  'p_barren', 'lake_frontage','p_crops', 'p_wet']
input_list_info = ['p_prot_2000_200','p_prot_2010_200','p_prot_1990_200',
                   'p_prot_2000_1000','p_prot_2010_5000','p_prot_1990_1000',
                   'p_prot_2000_5000','p_prot_2010_5000','p_prot_1990_5000',
                  'hh_inc_med_bg_1990', 'hh_inc_med_bg_2000','hh_inc_med_bg_2006-2010', 'hh_inc_med_bg_2012-2016',
'hh_inc_med_tract_1990', 'hh_inc_med_tract_2000','hh_inc_med_tract_2006-2010', 'hh_inc_med_tract_2012-2016',
'hh_inc_avg_bg_1990', 'hh_inc_avg_bg_2000', 'hh_inc_avg_bg_2006-2010','hh_inc_avg_bg_2012-2016', 
'hh_inc_avg_tract_1990','hh_inc_avg_tract_2000', 'hh_inc_avg_tract_2006-2010','hh_inc_avg_tract_2012-2016',
                  'pop_dens_bg_1990', 'pop_dens_bg_2000','pop_dens_bg_2006-2010', 'pop_dens_bg_2012-2016',
'pop_dens_tract_1990','pop_dens_tract_2000', 'pop_dens_tract_2006-2010','pop_dens_tract_2012-2016']
input_list = ['year','fip_avg']+input_list_resource+input_list_info
#imp = validate_model(training,input_list)
#select_model(training,input_list)
validate_model_Kfold(training,input_list)

In [37]:
input_list = ['year',
 'pop_dens_tract_1990',
 'pop_dens_tract_2000',
 'p_forest_e',
 'p_prot_2000_1000',
 'p_shrub',
 'p_prot_2010_200',
 'pop_dens_bg_1990',
 'p_grassland',
 'p_prot_2000_5000',
 'p_prot_2000_200',
 'p_wet',
 'p_forest_d',
 'p_prot',
 'lake_frontage']

df = select_model(training,input_list)
df

,rmse,nt,md
0,1.143100,50.0,5.0
1,1.064622,50.0,8.0
2,1.040826,50.0,10.0
3,1.015044,50.0,20.0
4,1.009990,50.0,22.0
5,1.011748,50.0,25.0
6,1.151091,100.0,5.0
7,1.083705,100.0,8.0
8,1.038253,100.0,10.0
9,0.993735,100.0,20.0


In [13]:
#get the feature importance of the random forest model
input_list = ['year','fip_avg']+input_list_resource+input_list_info
df_imp = pd.DataFrame(input_list,imp,columns=['att'])
df_imp.reset_index(level=0, inplace=True)
df_imp.rename(columns = {'index':'imp_rate'}, inplace = True) 
df_imp = df_imp.sort_values(by=['imp_rate'],ascending=False)
att_list = df_imp['att'].tolist()
df_imp

,imp_rate,att
0,0.157552,year
52,0.109034,pop_dens_tract_1990
53,0.089051,pop_dens_tract_2000
10,0.076959,p_forest_e
26,0.046460,p_prot_2000_1000
8,0.044181,p_shrub
24,0.036912,p_prot_2010_200
48,0.028972,pop_dens_bg_1990
22,0.020658,p_wet
23,0.020513,p_prot_2000_200


In [14]:
att_list

['year',
 'pop_dens_tract_1990',
 'pop_dens_tract_2000',
 'p_forest_e',
 'p_prot_2000_1000',
 'p_shrub',
 'p_prot_2010_200',
 'pop_dens_bg_1990',
 'p_wet',
 'p_prot_2000_200',
 'p_grassland',
 'p_prot_2000_5000',
 'p_forest_d',
 'p_prot_1990_5000',
 'lake_frontage',
 'p_prot',
 'p_prot_1990_1000',
 'p_prot_2010_5000',
 'p_prot_2010_5000',
 'pop_dens_bg_2000',
 'hh_inc_med_bg_1990',
 'p_dev_open',
 'p_crops',
 'p_pasture',
 'p_wetland_w',
 'hh_inc_avg_bg_1990',
 'p_prot_1990_200',
 'pop_dens_bg_2012-2016',
 'p_f',
 'hh_inc_avg_bg_2000',
 'pop_dens_bg_2006-2010',
 'pop_dens_tract_2012-2016',
 'hh_inc_med_bg_2000',
 'hh_inc_med_bg_2006-2010',
 'p_dev_low',
 'hh_inc_avg_bg_2012-2016',
 'lake_importance',
 'hh_inc_avg_bg_2006-2010',
 'hh_inc_med_tract_2006-2010',
 'hh_inc_avg_tract_2006-2010',
 'pop_dens_tract_2006-2010',
 'p_forest_m',
 'hh_inc_med_bg_2012-2016',
 'hh_inc_med_tract_1990',
 'hh_inc_avg_tract_1990',
 'hh_inc_med_tract_2000',
 'hh_inc_avg_tract_2000',
 'hh_inc_med_tract_2012-

In [27]:
art

,mse,num_att
0,1.869927,1.0
1,1.795677,2.0
2,1.741440,3.0
3,1.619746,4.0
4,1.656368,5.0
5,1.580401,6.0
6,1.558481,7.0
7,1.481804,8.0
8,1.452430,9.0
9,1.439923,10.0


In [28]:
def validate_model_Kfold(df,input_list):
    mae = []
    for train_idx, test_idx in KFold(n_splits = 5, shuffle = True).split(df):
        X_train, X_test = df.iloc[train_idx],df.iloc[test_idx]
        y_train, y_test = df['log_price_per_ha'].iloc[train_idx],df['log_price_per_ha'].iloc[test_idx]
        RFmodel = RandomForestRegressor(n_estimators = 300, max_depth = 20) #tested for n from 100 to 500 with 50 as increment, 300 is the best. tested md, 20 is the best
        RFmodel = RFmodel.fit(X_train[input_list] , y_train)
        y_rf = RFmodel.predict(X_test[input_list])
        rf_mae = mean_absolute_error(y_test, y_rf)
        #rf_mae = np.sqrt(mean_squared_error(y_test,y_rf))
        #print(rf_mae)
        mae.append(rf_mae)
    print("The MAE score by K-fold is ",np.mean(mae))
    
    

In [17]:
def validate_model(df,input_list):
    tr = df.loc[df['e_sold']==0].copy()
    te = df.loc[df['e_sold']==1].copy()

    rf = RandomForestRegressor(n_estimators=300)
    rf.fit(tr[input_list],tr['log_price_per_ha'])

    te['predicted'] = rf.predict(te[input_list])

    #mse = mean_absolute_error(te['log_price_per_ha'],te['predicted'])
    mse = np.sqrt(mean_squared_error(te['Score'],te['predicted']))
    print(mse)
    return rf.feature_importances_
    
    

In [25]:
#get the optimal combination of attributes that we should use in our model
def select_attribute(df,input_list):
    n_trails = 5
    att_list = []
    df_mse = pd.DataFrame(columns=['mse','num_att'])
    
    for i in input_list:
        mses = []
        att_list.append(i)
        for j in range(n_trails):
            tr = df.loc[df['e_sold']==0].copy()
            te = df.loc[df['e_sold']==1].copy()
            rf = RandomForestRegressor(n_estimators=300)
            rf.fit(tr[att_list],tr['log_price_per_ha'])
            te['predicted'] = rf.predict(te[att_list])
            mse = mean_absolute_error(te['log_price_per_ha'],te['predicted'])
            mses += [mse]
        df_mse.loc[df_mse.shape[0]] = [np.mean(mses),len(att_list)]
    
    
    return df_mse

In [35]:
#get the optimal parameter in the model
def select_model(df,input_list):
    
    df_rmse = pd.DataFrame(columns=['mae','nt','md'])
    
    for n_tree in [50,100,200,300]:
        for max_d in [5,8,10,20,22,25]:
            n_trial = 3
            mses = []


            for i in range(n_trial):
                
                tr = df.sample(10000).copy()
                te = df.loc[df['gisjoin'].isin(tr['gisjoin'])==False].sample(5000).copy()
                print('{}/{}\r'.format(i,n_trial),end='')
                
                rf = RandomForestRegressor(n_estimators=n_tree,max_depth=max_d)
                rf.fit(tr[input_list],tr['log_price_per_ha'])
                te['predicted'] = rf.predict(te[input_list])
                mse = mean_absolute_error(te['log_price_per_ha'],te['predicted'])
                mses += [mse]

            df_rmse.loc[df_rmse.shape[0]] = [np.mean(mses),n_tree,max_d]
            
    return df_rmse